In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

# 線形層による識別

In [45]:
from torchvision import datasets

dataset = datasets.MNIST('./mnistdata', train=True, download=True)

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
x0 = dataset.data.reshape(60000, -1) / 255.0
y0 = dataset.targets

In [26]:
y0.dtype

torch.int64

In [10]:
class MyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(784, 100)
        self.l2 = nn.Linear(100, 100)
        self.l3 = nn.Linear(100, 10)
        
    def forward(self, x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        return self.l3(h)

In [24]:
model = MyNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

In [25]:
n = len(y0)
bs = 200

for j in range(10):
    idx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = x0[idx[i:(i+bs) if (i+bs) < n else n]].to(device)
        y = y0[idx[i:(i+bs) if (i+bs) < n else n]].to(device)
        output = model(x)
        loss = criterion(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    outfile = 'nn-' + str(j) + '.model'
    torch.save(model.state_dict(), outfile)
    print(outfile, 'saved')

nn-0.model saved
nn-1.model saved
nn-2.model saved
nn-3.model saved
nn-4.model saved
nn-5.model saved
nn-6.model saved
nn-7.model saved
nn-8.model saved
nn-9.model saved


#### テスト

In [28]:
dataset = datasets.MNIST('./data', train=False, download=True)
xtest = dataset.data.reshape(10000, -1) / 255.0
ytest = dataset.targets

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [36]:
Xtest = xtest.to(device)
ytest = ytest.to(device)

model = MyNN()
model.load_state_dict(torch.load('nn-9.model'))
model.to(device)

MyNN(
  (l1): Linear(in_features=784, out_features=100, bias=True)
  (l2): Linear(in_features=100, out_features=100, bias=True)
  (l3): Linear(in_features=100, out_features=10, bias=True)
)

In [37]:
model.eval()

with torch.no_grad():
    y1 = model(Xtest)
    ans = torch.argmax(y1, 1)
    print(((ytest == ans).sum().float()/len(ans)).item())

0.9662999510765076


# CNNによる画像識別

#### nn.Conv2d(入力画像のチャンネル数, フィルタの数,　フィルタのサイズ )
#### nn.MaxPool2d(区分けする領域の大きさ)

In [61]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset = datasets.MNIST('./data', train=True, download=True)
x0 = dataset.data.reshape(60000,1,28,28) / 255.0
y0 = dataset.targets


In [68]:
class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cn1 = nn.Conv2d(1, 20, 5)
        self.pool1 = nn.MaxPool2d(2)
        self.cn2 = nn.Conv2d(20, 50, 5)
        self.fc = nn.Linear(3200, 10)
        
    def forward(self, x):
        x = F.relu(self.cn1(x))
        x = self.pool1(x)
        x = F.relu(self.cn2(x))
        x = x.reshape(len(x), -1)
        print(x.shape)
        return self.fc(x)

In [69]:
model = MyCNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [70]:
n = len(y0) 
bs = 200

model.train()
for j in range(10):
    idx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = x0[idx[i:(i+bs) if (i+bs) < n else n]].to(device)
        y = y0[idx[i:(i+bs) if (i+bs) < n else n]].to(device)                
        output = model(x)
        loss = criterion(output,y)        
        print(j, i, loss.item())
        optimizer.zero_grad()                
        loss.backward()
        optimizer.step()
        
    outfile = "cnn0-" + str(j) + ".model"
    torch.save(model.state_dict(),outfile) 
    print(outfile," saved")



torch.Size([200, 3200])
0 0 2.308659076690674
torch.Size([200, 3200])
0 200 2.3041954040527344
torch.Size([200, 3200])
0 400 2.2902839183807373
torch.Size([200, 3200])
0 600 2.279113292694092
torch.Size([200, 3200])
0 800 2.2861344814300537
torch.Size([200, 3200])
0 1000 2.2588412761688232
torch.Size([200, 3200])
0 1200 2.2440567016601562
torch.Size([200, 3200])
0 1400 2.2306015491485596
torch.Size([200, 3200])
0 1600 2.2095839977264404
torch.Size([200, 3200])
0 1800 2.1832494735717773
torch.Size([200, 3200])
0 2000 2.168170690536499
torch.Size([200, 3200])
0 2200 2.1440844535827637
torch.Size([200, 3200])
0 2400 2.105591058731079
torch.Size([200, 3200])
0 2600 2.0433311462402344
torch.Size([200, 3200])
0 2800 1.999216914176941
torch.Size([200, 3200])
0 3000 1.926728367805481
torch.Size([200, 3200])
0 3200 1.9056739807128906
torch.Size([200, 3200])
0 3400 1.8098846673965454
torch.Size([200, 3200])
0 3600 1.7174148559570312
torch.Size([200, 3200])
0 3800 1.6177582740783691
torch.Size([2

In [56]:


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
import numpy as np

# GPU/CPU device setting

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data setting

dataset = datasets.MNIST('./data', train=True, download=True)
x0 = dataset.data.reshape(60000,1,28,28) / 255.0
y0 = dataset.targets

# Define model

class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.cn1 = nn.Conv2d(1, 20, 5)  
        self.pool1 = nn.MaxPool2d(2)    
        self.cn2 = nn.Conv2d(20, 50, 5) 
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(50 * 8 * 8, 10)
    def forward(self, x):
        x = F.relu(self.cn1(x))
        x = self.pool1(x)
        x = F.relu(self.cn2(x))
#        x = self.dropout(x)
        x = x.view(len(x), -1) # Flatten
        return self.fc(x)

# Initialize model

model = MyCNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Learn

n = len(y0) 
bs = 200

model.train()
for j in range(10):
    idx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = x0[idx[i:(i+bs) if (i+bs) < n else n]].to(device)
        y = y0[idx[i:(i+bs) if (i+bs) < n else n]].to(device)                
        output = model(x)
        loss = criterion(output,y)        
        print(j, i, loss.item())
        optimizer.zero_grad()                
        loss.backward()
        optimizer.step()
        
    outfile = "cnn0-" + str(j) + ".model"
    torch.save(model.state_dict(),outfile) 
    print(outfile," saved")

/home/takakiyuto/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0 0 2.3133339881896973
0 200 2.2998046875
0 400 2.3035144805908203
0 600 2.2921628952026367
0 800 2.274506092071533
0 1000 2.2623140811920166
0 1200 2.237166404724121
0 1400 2.217883825302124
0 1600 2.194443702697754
0 1800 2.188826560974121
0 2000 2.156815528869629
0 2200 2.1334095001220703
0 2400 2.1091299057006836
0 2600 2.0527405738830566
0 2800 1.9597872495651245
0 3000 1.9035170078277588
0 3200 1.877714991569519
0 3400 1.7976083755493164
0 3600 1.6614623069763184
0 3800 1.6034616231918335
0 4000 1.4527390003204346
0 4200 1.3100205659866333
0 4400 1.1749988794326782
0 4600 1.0878676176071167
0 4800 1.039279580116272
0 5000 0.824726939201355
0 5200 0.7470038533210754
0 5400 0.7446081042289734
0 5600 0.7708579301834106
0 5800 0.563111424446106
0 6000 0.6667057275772095
0 6200 0.7536942958831787
0 6400 0.5543975830078125
0 6600 0.5536776185035706
0 6800 0.47431424260139465
0 7000 0.5393960475921631
0 7200 0.4770994484424591
0 7400 0.4062362313270569
0 7600 0.5717474222183228
0 7800 0